# Youtube chatbot

## Import relevant modules

In [64]:
#from langchain.document_loaders import YoutubeLoader
from langchain_community.document_loaders import YoutubeLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS

# from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from dotenv import find_dotenv, load_dotenv
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.runnables.base import RunnableSequence

import textwrap

## Creating the database

In [2]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

In [3]:
video_url = "https://www.youtube.com/watch?v=vetLEg83xEA&list=PLaN_KBqoRXfgyytH4hqPBTwO6YIflCoTq&index=3&ab_channel=LaylaMartin"

In [4]:
loader = YoutubeLoader.from_youtube_url(video_url)
loader

In [5]:
transcript = loader.load()
transcript

[Document(metadata={'source': 'vetLEg83xEA'}, page_content="anything in life can be medicine or poison right and so instead of just making anger poison how do you make it medicine and the way that you relate to it in the way that you experience it and ultimately you know one of the things that healed me was being able to have an anger gasm like literally being able to orgasm with the energy of my anger it so revolutionized my relationship to this emotion yes absolutely the first time I had a shame gasm was one of the most spiritual experiences I've I've ever had yeah you know it was like whoa this is shame yeah like this is my shame yeah you know and and to me it changed everything once we've worked on the charge around that past stuff we still need to meet these emotions inside oursel as a human being and that presence piece is is so incredibly key feeling that sense of Honor in being able to feel this in the here and now it just makes it so much more digestible I think in that way we

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
text_splitter

In [7]:
docs = text_splitter.split_documents(transcript)
docs

[Document(metadata={'source': 'vetLEg83xEA'}, page_content="anything in life can be medicine or poison right and so instead of just making anger poison how do you make it medicine and the way that you relate to it in the way that you experience it and ultimately you know one of the things that healed me was being able to have an anger gasm like literally being able to orgasm with the energy of my anger it so revolutionized my relationship to this emotion yes absolutely the first time I had a shame gasm was one of the most spiritual experiences I've I've ever had yeah you know it was like whoa this is shame yeah like this is my shame yeah you know and and to me it changed everything once we've worked on the charge around that past stuff we still need to meet these emotions inside oursel as a human being and that presence piece is is so incredibly key feeling that sense of Honor in being able to feel this in the here and now it just makes it so much more digestible I think in that way we

In [8]:
db = FAISS.from_documents(docs, embeddings)
db

### Wrap it up in a function

In [9]:
def create_db_from_youtube_video_url(video_url):
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)

    db = FAISS.from_documents(docs, embeddings)
    return db

In [10]:
db = create_db_from_youtube_video_url(video_url)
db

## Get response from query

In [12]:
query = "what is this video about?"

docs = db.similarity_search(query, k=4)
docs

[Document(metadata={'source': 'vetLEg83xEA'}, page_content="billion industry by the year 2026 why is this because people are more ready than ever to express their sexuality and build deeply connected meaningful relationships and also obviously to have the most outrageous orgasms and erotic experiences of their lives but most people are never taught how to do any of this how to have extraordinary relationships how to have the sex they really crave they're looking for qualified confident professionals to teach them and my Vita sex love and relationship coaching certification is a powerful one-year professional certification that's helped more than 2,000 coaches make their way to becoming incredibly successful and highly paid sex love and relationship professionals but this program doesn't just train you in every aspect of the coaching process so that you can build a successful online business eita coaching will also walk you through the personal transformation of a lifetime because the b

In [13]:
docs_page_content = " ".join([d.page_content for d in docs])
docs_page_content

"billion industry by the year 2026 why is this because people are more ready than ever to express their sexuality and build deeply connected meaningful relationships and also obviously to have the most outrageous orgasms and erotic experiences of their lives but most people are never taught how to do any of this how to have extraordinary relationships how to have the sex they really crave they're looking for qualified confident professionals to teach them and my Vita sex love and relationship coaching certification is a powerful one-year professional certification that's helped more than 2,000 coaches make their way to becoming incredibly successful and highly paid sex love and relationship professionals but this program doesn't just train you in every aspect of the coaching process so that you can build a successful online business eita coaching will also walk you through the personal transformation of a lifetime because the best coaches have done the work themselves you'll discover s

In [17]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.2)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115f518d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115f53a00>, root_client=<openai.OpenAI object at 0x1154fe0e0>, root_async_client=<openai.AsyncOpenAI object at 0x115f51690>, model_name='gpt-3.5-turbo-16k', temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [20]:
# Template to use for the system message prompt
template = """
    You are a helpful assistant that that can answer questions about youtube videos 
    based on the video's transcript: {docs}

    Only use the factual information from the transcript to answer the question.

    If you feel like you don't have enough information to answer the question, say "I don't know".

    """

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that that can answer questions about youtube videos \n    based on the video\'s transcript: {docs}\n\n    Only use the factual information from the transcript to answer the question.\n\n    If you feel like you don\'t have enough information to answer the question, say "I don\'t know".\n\n    '), additional_kwargs={})

In [21]:
# Human question prompt
human_template = "Answer the following question: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
human_message_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Answer the following question: {question}'), additional_kwargs={})

In [22]:
chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

chat_prompt

ChatPromptTemplate(input_variables=['docs', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that that can answer questions about youtube videos \n    based on the video\'s transcript: {docs}\n\n    Only use the factual information from the transcript to answer the question.\n\n    If you feel like you don\'t have enough information to answer the question, say "I don\'t know".\n\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Answer the following question: {question}'), additional_kwargs={})])

In [46]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain

LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['docs', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that that can answer questions about youtube videos \n    based on the video\'s transcript: {docs}\n\n    Only use the factual information from the transcript to answer the question.\n\n    If you feel like you don\'t have enough information to answer the question, say "I don\'t know".\n\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Answer the following question: {question}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115f518d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115f53a00>, root_c

In [47]:
response = chain.run(question=query, docs=docs_page_content)
response = response.replace("\n", "")
response

'This video is about the Vita Sex, Love, and Relationship Coaching Certification program, which aims to teach people how to have extraordinary relationships, express their sexuality, and become successful and highly paid professionals in the sex, love, and relationship industry. The video also mentions various transformative techniques and practices that are taught in the program, as well as the opportunity to attend an exclusive live embodiment retreat.'

In [48]:
chain = chat_prompt | chat
chain

ChatPromptTemplate(input_variables=['docs', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that that can answer questions about youtube videos \n    based on the video\'s transcript: {docs}\n\n    Only use the factual information from the transcript to answer the question.\n\n    If you feel like you don\'t have enough information to answer the question, say "I don\'t know".\n\n    '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Answer the following question: {question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115f518d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115f53a00>, root_client=<openai.OpenAI object at 0x1

In [49]:
response = chain.invoke({'question':query, 'docs':docs_page_content})
response

AttributeError: 'AIMessage' object has no attribute 'replace'

### Wrap it up in a function

In [54]:
def get_response_from_query(db, query, k=4):
    docs = db.similarity_search(query, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])

    chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.2)

    # Template to use for the system message prompt
    template = """
        You are a helpful assistant that that can answer questions about youtube videos 
        based on the video's transcript: {docs}
        
        Only use the factual information from the transcript to answer the question.
        
        If you feel like you don't have enough information to answer the question, say "I don't know".
        
        """

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    # Human question prompt
    human_template = "Answer the following question: {question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

#     chain = LLMChain(llm=chat, prompt=chat_prompt)

#     response = chain.run(question=query, docs=docs_page_content)
#     response = response.replace("\n", "")

    chain = chat_prompt | chat
    response = chain.invoke({'question':query, 'docs':docs_page_content})

    return response, docs

In [55]:
response, docs = get_response_from_query(db, query, k=4)

In [58]:
response.content

'This video is about the Vita Sex, Love, and Relationship Coaching Certification program, which aims to train individuals to become successful and highly paid professionals in the sex, love, and relationship coaching industry. The program focuses on teaching techniques and tools for building extraordinary relationships, having fulfilling sex lives, and experiencing transformative personal growth. It also mentions a live embodiment retreat for students and alumni of the program.'

In [57]:
docs

[Document(metadata={'source': 'vetLEg83xEA'}, page_content="billion industry by the year 2026 why is this because people are more ready than ever to express their sexuality and build deeply connected meaningful relationships and also obviously to have the most outrageous orgasms and erotic experiences of their lives but most people are never taught how to do any of this how to have extraordinary relationships how to have the sex they really crave they're looking for qualified confident professionals to teach them and my Vita sex love and relationship coaching certification is a powerful one-year professional certification that's helped more than 2,000 coaches make their way to becoming incredibly successful and highly paid sex love and relationship professionals but this program doesn't just train you in every aspect of the coaching process so that you can build a successful online business eita coaching will also walk you through the personal transformation of a lifetime because the b

## Example usage

In [59]:
video_url = "https://www.youtube.com/watch?v=vetLEg83xEA&list=PLaN_KBqoRXfgyytH4hqPBTwO6YIflCoTq&index=3&ab_channel=LaylaMartin"
db = create_db_from_youtube_video_url(video_url)

In [60]:
query = "what do they say abouth a healty way to express anger and rage?"
response, docs = get_response_from_query(db, query)

In [62]:
response.content

'They say that a healthy way to express anger and rage is to allow oneself to feel and experience the emotions fully, rather than suppressing them. They also mention the importance of setting boundaries, using anger to establish safety, and choosing when to use anger to protect oneself. Additionally, they talk about the connection between anger and power, and how expressing anger can help restore boundaries and reclaim personal power. They also mention the concept of an "anger gasm," which involves integrating the energy of anger into a pleasurable or orgasmic state.'